In [387]:
import os
import numpy as np
import pandas as pd
from collections import deque
!pip install music21
from music21 import converter, instrument, note, chord, midi, stream
from keras.models import Model, Sequential
from keras.layers import *
from keras.models import load_model
import keras.backend as K
import keras.callbacks
from keras.utils import Sequence
from keras.utils import to_categorical
from keras.preprocessing.sequence import TimeseriesGenerator

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [508]:
notes = []
durations = []
track = 0
scale = 1

directory = "midi/Guitar_Sor"
for i, file in enumerate(os.listdir(directory)):
    midi_part = converter.parse(os.path.join(directory, file))
    # Use only first track
    midi_part = midi_part[track]
    notes_to_parse = None
    # Parse the midi file by the notes/chords it contains
    notes_to_parse = midi_part.flat.notesAndRests
    for elem in notes_to_parse:
        if isinstance(elem, note.Note):
            notes.append([str(elem.pitch)])
        elif isinstance(elem, chord.Chord):
            notes.append([str(n.nameWithOctave) for n in elem.pitches])
        elif isinstance(elem, note.Rest):
            notes.append([elem.name])
        durations.append(elem.quarterLength)
    
    notes.append(["rest"])
    durations.append(1.0)
    print("Song %s Loaded" % file)
                
print("DONE LOADING SONGS")    
# Get all pitch names
np_notes = np.array(notes)
np_notes = np_notes.flatten()
notes_vocab = []
for item in np_notes:
    if len(item) > 1 and item != "rest":
        notes_vocab.append(",".join([note for note in item]))
    else:
        notes_vocab.append(item[0])
notes_vocab = sorted(set(notes_vocab))
# Get all durations
durations_vocab = []
durations_vocab = sorted(set(durations))

notes_vocab_len = len(notes_vocab)
durations_vocab_len = len(durations_vocab)
number_notes = len(notes)
print(notes_vocab_len, durations_vocab_len, number_notes)
print(notes_vocab, notes)
durations_vocab_len = len(durations_vocab)
print(durations_vocab, durations)

Song Sor_Study_no1.mid Loaded
Song Sor_Study_no10.mid Loaded
Song Sor_Study_no11.mid Loaded
Song Sor_Study_no12.mid Loaded
Song Sor_Study_no13.mid Loaded
Song Sor_Study_no14.mid Loaded
Song Sor_Study_no15.mid Loaded
Song Sor_Study_no16.mid Loaded
Song Sor_Study_no17.mid Loaded
Song Sor_Study_no18.mid Loaded
Song Sor_Study_no19.mid Loaded
Song Sor_Study_no2.mid Loaded
Song Sor_Study_no20a.mid Loaded
Song Sor_Study_no3.mid Loaded
Song Sor_Study_no4.mid Loaded
Song Sor_Study_no5.mid Loaded
Song Sor_Study_no6.mid Loaded
Song Sor_Study_no7.mid Loaded
Song Sor_Study_no8.mid Loaded
Song Sor_Study_no9.mid Loaded
DONE LOADING SONGS
387 21 6892
['A2', 'A3', 'A3,A2', 'A3,B2', 'A3,B3', 'A3,C#3', 'A3,C3,A2', 'A3,C4', 'A3,C4,F#4', 'A3,C4,G4', 'A3,D3', 'A3,D4,F#4', 'A3,E-3,F2', 'A3,E3', 'A3,E3,C3', 'A3,F#3', 'A3,F#3,B2', 'A3,F3', 'A3,F4', 'A4', 'A4,A2', 'A4,A3', 'A4,B3', 'A4,C#4', 'A4,C#4,A2', 'A4,C#4,A3', 'A4,C#4,A3,A2', 'A4,C4', 'A4,C4,A2', 'A4,C4,E-3', 'A4,C5', 'A4,D4', 'A4,E-4,C4', 'A4,E-4,C4,F3'

In [509]:
# turn notes to integers:
cat_notes = []
for elem in notes:
    int_note = notes_vocab.index(",".join(elem))
    cat = np.zeros((len(notes_vocab)))
    cat[int_note] = 1
    cat_notes.append(cat)
cat_durations = []
for elem in durations:
    int_duration = durations_vocab.index(elem)
    cat = np.zeros((len(durations_vocab)))
    cat[int_duration] = 1
    cat_durations.append(cat)

# merge
x = [np.array(cat_notes), np.array(cat_durations)]

In [510]:
seq_len = 32 * scale
batch_size = 32
split = int(0.8 * len(x))


class dataGenerator(Sequence):
    def __init__(self, x, batch_size):
        self.batch_size = batch_size
        self.x = x
    def __len__(self):
        return len(self.x[0]) // self.batch_size - 1
    def __getitem__(self, idx):
        X_note = []
        X_duration = []
        Y_note = []
        Y_duration = []
        for i in range(self.batch_size):
            X_note.append(self.x[0][idx+i : idx+i + seq_len])
            X_duration.append(self.x[1][idx+i : idx+i + seq_len])
            Y_note.append(self.x[0][idx+i + seq_len])
            Y_duration.append(self.x[1][idx+i + seq_len])
        return [np.array(X_note), np.array(X_duration)], [np.array(Y_note), np.array(Y_duration)]
data_gen = dataGenerator(x, batch_size)

In [512]:
print(x[0].shape, x[1].shape)
print(i, data_gen[0][0][0].shape, data_gen[0][1][0].shape)

(6892, 387) (6892, 21)
214 (32, 32, 387) (32, 387)


In [453]:
# Build Model

In [513]:
K.clear_session()

in_note = Input(shape=(seq_len, len(notes_vocab)))
x1 = Bidirectional(LSTM(32 * scale))(in_note)

in_duration = Input(shape=(seq_len, len(durations_vocab)))
x2 = Bidirectional(LSTM(32 * scale))(in_duration)

out_note = Dense(len(notes_vocab), activation='softmax')(x1)
out_duration = Dense(len(durations_vocab), activation='softmax')(x2)

model = Model(inputs=[in_note, in_duration], outputs=[out_note, out_duration])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 387)      0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32, 21)       0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 64)           107520      input_1[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 64)           13824       input_2[0][0]                    
__________________________________________________________________________________________________
dense_1 (D

In [514]:
#save best model if model improved
model_name = "Sor_multi.h5"
best_checkpoint = keras.callbacks.ModelCheckpoint(model_name, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [515]:
h = model.fit_generator(data_gen, epochs = 100, callbacks=[best_checkpoint])

Epoch 1/100
214/214 [==============================] - 9s 44ms/step - loss: 4.8113 - dense_1_loss: 3.2020 - dense_2_loss: 1.6093 - dense_1_acc: 0.2848 - dense_2_acc: 0.5000
Epoch 2/100
214/214 [==============================] - 8s 36ms/step - loss: 3.1945 - dense_1_loss: 2.4356 - dense_2_loss: 0.7589 - dense_1_acc: 0.3048 - dense_2_acc: 0.7897
Epoch 3/100
214/214 [==============================] - 8s 38ms/step - loss: 2.7117 - dense_1_loss: 2.2717 - dense_2_loss: 0.4400 - dense_1_acc: 0.3058 - dense_2_acc: 0.9003
Epoch 4/100
214/214 [==============================] - 8s 40ms/step - loss: 2.3723 - dense_1_loss: 2.0561 - dense_2_loss: 0.3162 - dense_1_acc: 0.3614 - dense_2_acc: 0.9154
Epoch 5/100
214/214 [==============================] - 9s 40ms/step - loss: 2.0864 - dense_1_loss: 1.8402 - dense_2_loss: 0.2462 - dense_1_acc: 0.4629 - dense_2_acc: 0.9343
Epoch 6/100
214/214 [==============================] - 9s 40ms/step - loss: 1.7904 - dense_1_loss: 1.5832 - dense_2_loss: 0.2072 - dens

KeyboardInterrupt: 

In [ ]:
#print History graph
historydf = pd.DataFrame(h.history, index=h.epoch)
historydf.plot(ylim=(0,1))

In [ ]:
# Load / save

In [372]:
model.save("Sor_time.h5")
#model = load_model("Sor.h5")

In [562]:
# Build seed
idx = 5
x_test, _ = data_gen[idx]
x_test = [x_test[0][0], x_test[1][0]]
x_test_stream = stream.Stream()

for i in range(seq_len):
    str_note = notes_vocab[np.argmax(x_test[0][i])]
    duration = durations_vocab[np.argmax(x_test[1][i])]
    if len(str_note.split(",")) > 1:
        _chord = chord.Chord(str_note.split(","))
        _chord.quarterLength = duration
        x_test_stream.append(_chord)
    else:
        if str_note != "rest":
            _note = note.Note(str_note)
            _note.quarterLength = duration
            x_test_stream.append(_note)
        else:
            _rest = note.Rest()
            _rest.quarterLength = duration
            x_test_stream.append(_rest)
x_test_stream.insert(0, instrument.Guitar())
print(x_test[0].shape, x_test[1].shape)

(32, 387) (32, 21)


In [563]:
# make seq_len predictions from seed
preds = []
x = x_test

for i in range(seq_len):
    _note, _duration = model.predict([np.array([x[0]]), np.array([x[1]])])
    
    cat_note = np.zeros((len(notes_vocab)))
    _note = np.argmax(_note)
    cat_note[_note] = 1
    
    cat_duration = np.zeros((len(durations_vocab)))
    _duration = np.argmax(_duration)
    cat_duration[_duration] = 1
    
    x[0] = x[0][1:]
    x[0] = list(x[0]) + [cat_note]
    x[1] = x[1][1:]
    x[1] = list(x[1]) + [cat_duration]
    preds.append((cat_note, cat_duration))
print(len(preds), [(np.argmax(pred[0]), np.argmax(pred[1])) for pred in preds])

32 [(224, 14), (386, 3), (163, 10), (106, 10), (19, 10), (357, 10), (357, 0), (290, 14), (386, 3), (224, 10), (163, 10), (19, 10), (357, 10), (357, 0), (290, 14), (386, 3), (226, 10), (224, 10), (163, 10), (127, 10), (89, 10), (133, 10), (174, 10), (242, 15), (386, 10), (133, 10), (173, 10), (242, 10), (300, 18), (179, 10), (242, 10), (300, 10)]


In [564]:
# Build predicted stream
y_test_stream = stream.Stream()
for i in range(len(preds)):
    str_note = notes_vocab[np.argmax(preds[i][0])]
    duration = durations_vocab[np.argmax(preds[i][1])]
    if len(str_note.split(",")) > 1:
        _chord = chord.Chord(str_note.split(","))
        _chord.quarterLength = duration
        y_test_stream.append(_chord)
    else:
        if str_note != "rest":
            _note = note.Note(str_note)
            _note.quarterLength = duration
            y_test_stream.append(_note)
        else:
            _rest = note.Rest()
            _rest.quarterLength = duration
            y_test_stream.append(_rest)
y_test_stream.insert(0, instrument.Guitar())

In [565]:
# save streams to midi files
x_test_stream.write("midi", "x_test.mid"), y_test_stream.write("midi", "y_test.mid")

('x_test.mid', 'y_test.mid')

In [566]:
# play seed
sp = midi.realtime.StreamPlayer(x_test_stream)
sp.play()

In [567]:
# play generated music
sp = midi.realtime.StreamPlayer(y_test_stream)
sp.play()